In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import os
import re
import math
import functools
import traceback
import seaborn as sb
import matplotlib.pyplot as plt
import anndata
import scanpy as sc
import spatialdata_plot
import numpy as np
import pandas as pd
import corescpy as cr

# Main
write_object = True  # change to True when you're ready to save objects
overwrite = False  # overwrite if already exists?
col_cell_type = "leiden_res1pt5_dist0_npc30"
col_ann = "Bucket"
regions = ["mucosa", "serosa", "myenteric_plexus",
           "submucosa", "smc_circular", "smc_longitudinal"]

# Process Options
panel = "TUQ97N"  # Xenium panel ID
constants_dict = cr.get_panel_constants(panel_id=panel)
libs = [  # sample IDs from patients for whom we have all conditions
    "50452A", "50452B", "50452C",  # old segmentation
    "50006A", "50006B", "50006C",  # rest are new segmentation
    "50217A", "50217B", "50217C",
    "50336B", "50336C", "50336A",
    "50403A2", "50403B", "50403C1"
]  # excludes low-quality sample/condition replicates 50403A1 & 50403C2
# libs = None  # to run all available samples
cols = [
    "transcript_counts", "cell_area", "nucleus_area", "nucleus_count",
    "control_probe_counts", "control_codeword_counts",
    "unassigned_codeword_counts", "deprecated_codeword_counts"
]
# cols_plot = list(cols)
cols_plot = ["transcript_counts", "cell_area", "nucleus_area",
             "nucleus_count", "control_probe_counts"]
input_suffix = ""  # in case want to load objects with some suffix

# Files & Directories
direc = "/mnt/cho_lab/bbdata2/"  # mounted NFS with data
dir_entry = "/mnt/cho_lab/disk2"  # Spark writeable data directory
mdf = str("/mnt/cho_lab/disk2/elizabeth/data/shared-xenium-library/samples_"
          f"{panel}.csv")  # metadata file path (for now; will soon be on NFS)
dir_writeable = os.path.join(
    dir_entry, f"elizabeth/data/shared-xenium-library")  # where objects are
out_dir = os.path.join(
    dir_writeable, f"outputs/{panel}/nebraska")  # object output directory

#  Your Folders
out_new = os.path.join(
    dir_entry,
    f"{os.getlogin()}/data/shared-xenium-library/outputs/{panel}/nebraska")

# Constants (Shouldn't Need Edits Unless Extreme Process Changes)
cso, col_sample, col_condition, col_inflamed, col_subject = [
    constants_dict[x] if x in constants_dict else None for x in [
        "col_sample_id_o", "col_sample_id", "col_condition",
        "col_inflamed", "col_subject"]]
dir_data = os.path.join(direc, f"outputs/{panel}")
files = functools.reduce(lambda i, j: i + j, [[os.path.join(
    run, i) for i in os.listdir(os.path.join(
        dir_data, run))] for run in os.listdir(dir_data)])  # all data paths
os.makedirs(out_dir, exist_ok=True)  # make output directory if needed
metadata = cr.pp.get_metadata_cho(direc, mdf, panel_id=panel, samples=libs)
f_ann = pd.read_excel(os.path.join(
    out_dir, "annotation_dictionaries/annotations_all.xlsx"), index_col=[
        0, 1]).dropna(how="all").dropna(how="all", axis=1)
f_ann = f_ann.reset_index().astype({f_ann.index.names[1]: "int"}).astype({
    f_ann.index.names[1]: "string"}).set_index(f_ann.index.names)
metadata[col_subject]

ryp2 is not installed. Install with pip install rpy2 to run tools with R support.

Sample
Uninflamed-50452A     50452
Inflamed-50452B       50452
Stricture-50452C      50452
Inflamed-50006A       50006
Uninflamed-50006B     50006
Stricture-50006C      50006
Inflamed-50217A       50217
Uninflamed-50217B     50217
Stricture-50217C      50217
Inflamed-50336B       50336
Uninflamed-50336C     50336
Stricture-50336A      50336
Uninflamed-50403A2    50403
Inflamed-50403B       50403
Stricture-50403C1     50403
Name: subject_id, dtype: int64

# Overall Cell Counts & Metadata

In [ ]:
n_origin, n_proc, obs = {}, {}, {}
for s in libs:  # iterate samples
    print(f"\n\n{'=' * 80}\n{s}\n{'=' * 80}\n\n")
    fff = os.path.join(dir_data, np.array(files)[np.where([
        s == os.path.basename(x).split("__")[2].split("-")[0]
        for x in files])[0][0]])  # sample's Xenium data directory path
    lib = metadata.reset_index().set_index(cso).loc[s][col_sample]
    file_obj_proc = os.path.join(out_dir, f"{lib}{input_suffix}.h5ad")
    self = cr.Spatial(fff, library_id=lib)  # load original data
    adata = sc.read_h5ad(file_obj_proc)  # processed adata
    n_origin[s], n_proc[s] = self.rna.obs.shape[0], adata.obs.shape[0]
    obs[s] = adata.obs[adata.obs.columns.intersection(set(cols + [
        col_cell_type]))].groupby(col_cell_type).describe().copy()
    sb.pairplot(adata.obs[adata.obs.columns.intersection(set(cols))])
n_cells = pd.concat([pd.Series(x, index=pd.Index(libs, name=cso))
                     for x in [n_origin, n_proc]],
                    keys=["Original", "Processed"], names=["Source"], axis=1)
n_cells = n_cells.assign(Percent=100 * n_cells["Processed"] / n_cells[
    "Original"])  # percent of cells remaining after processing
obs = pd.concat(obs, keys=libs, names=[cso])

## Write

In [ ]:
# n_cells.to_csv(os.path.join(
#     out_new, f"quantification/xenium_n_cells{input_suffix}.csv"))
# obs.to_csv(os.path.join(
#     out_new, f"quantification/xenium_qc{input_suffix}.csv"))

## Reload

In [ ]:
# n_cells = pd.read_csv(os.path.join(
#     out_new, f"quantification/xenium_n_cells{input_suffix}.csv"),
#                       index_col=0)
# obs = pd.read_csv(os.path.join(
#     out_new, f"quantification/xenium_qc{input_suffix}.csv"),
#                   header=[0, 1], index_col=[0, 1]).rename_axis([
#                       "Variable", "Metric"], axis=1).stack(0)

# Inspect

In [ ]:
n_cells

In [ ]:
obs

## Plot

In [ ]:
palette = "tab20"
# palette = ["r", "b", "y"]

fig = sb.catplot(obs[["25%", "50%", "75%"]].stack().to_frame("Value").join(
    metadata.reset_index().set_index(cso)), x=col_subject, y="Value",
                 row="Variable", col="Metric", kind="bar", hue=col_condition,
                 sharex=False, sharey=False, palette=palette)
fig.set_xticklabels(rotation=45, fontsize=10)
plt.subplots_adjust(hspace=0.3)

# By Region

In [ ]:
adata.obs.columns

In [ ]:
col_cell_type = "leiden_res1pt5_dist0_npc30"

obs_reg, n_cells_reg = {},{}
for r in regions:
    obs_reg[r], n_cells_reg[r] = {},{}
    for s in libs:  # iterate samples
        print(f"\n\n{'=' * 80}\n{s}\n{'=' * 80}\n\n")
        fff = os.path.join(dir_data, np.array(files)[np.where([
            s == os.path.basename(x).split("__")[2].split("-")[0]
            for x in files])[0][0]])  # sample's Xenium data directory path
        lib = metadata.reset_index().set_index(cso).loc[s][col_sample]
        file_obj_proc = os.path.join(out_dir, "objects_cropped",
                                     r, f"{lib}_{r}{input_suffix}.h5ad")
        if not os.path.exists(file_obj_proc):
            print(f"{file_obj_proc} doesn't exist")
            continue
        adata = sc.read_h5ad(file_obj_proc)  # processed adata
        obs_reg[r][s] = adata.obs[adata.obs.columns.intersection(set(cols + [
            col_cell_type]))].assign(n_cells=adata.obs.shape[0])
        n_cells_reg[r][s] = adata.obs[col_cell_type].value_counts()
        # adata.obs = adata.obs.join(
        #     adata.obs[col_cell_type].astype(str).map(dict(f_ann.loc[
        #         f"{lib}___{col_cell_type}_dictionary.xlsx"])[
        #             "bucket"]).to_frame("Annotation"),
        #     lsuffix="_original")
        # n_cells_reg[r][s] = adata.obs["Annotation"].value_counts()
    n_cells_reg[r] = pd.concat(n_cells_reg[r], names=[cso])
    obs_reg[r] = pd.concat(obs_reg[r], names=[cso])
obs_reg = pd.concat(obs_reg, names=["Region"])
n_cells_reg = pd.concat(n_cells_reg, names=["Region"])
n_cells_reg = n_cells_reg.to_frame("Count").join(n_cells_reg.groupby([
    "Region", "sample_id"]).sum().to_frame("Total"))
n_cells_reg.loc[:, "Percent"] = 100 * n_cells_reg[
    "Count"] / n_cells_reg["Total"]

## Write

In [ ]:
files = [os.path.join(
    out_new, f"quantification/xenium_qc_{'_'.join(regions)}{input_suffix}"
    ".csv"), os.path.join(
        out_new, f"quantification/xenium_n_cells_{'_'.join(regions)}"
        f"{input_suffix}.csv")]
print(files)
obs_reg.to_csv(files[0])
n_cells_reg.to_csv(files[1])

## Reload

In [ ]:
obs_reg = pd.read_csv(os.path.join(
    out_new, f"quantification/xenium_qc_{'_'.join(regions)}{input_suffix}"
    ".csv"), index_col=0)
n_cells_reg = pd.read_csv(os.path.join(
    out_new, f"quantification/xenium_n_cells_{'_'.join(regions)}"
    f"{input_suffix}.csv"), index_col=[0, 1, 2])

## Inspect

In [ ]:
n_cells_reg

In [ ]:
obs_reg

## Plot

## By Region

In [ ]:
obs_reg = obs_reg.join(metadata.reset_index().set_index(cso))
n_cells_reg = n_cells_reg.join(metadata.reset_index().set_index(cso))
sb.catplot(n_cells_reg, x=col_cell_type, y="Percent", hue="Region",
           kind="bar", col=col_condition, row=col_subject)
fig = sb.pairplot(obs_reg.reset_index()[cols_plot + ["Region"]], hue="Region",
                  kind="scatter", diag_kind="kde",
                  diag_kws=dict(cut=0, fill=True))

## By Condition & Region

In [ ]:
for r in obs_reg.reset_index().Region.unique():
    fig = sb.pairplot(obs_reg.loc[r][cols_plot], hue=col_condition,
                      kind="scatter", diag_kind="kde",
                      diag_kws=dict(cut=0, fill=True))
    fig.fig.suptitle(r)